# dataframe 내의 컬럼의 type 명시적 지정방식 정리

### 목표
dataframe내의 특정 컬럼의 데이터 타입을 명시적으로 지정하는 방식을 정리한다. REST API에서 오는 데이터는 대부분 VALUE가 String이다. 따라서 insert시 형변환을 해주어야 한다.  

여기서는 이런 이유로 dataframe에 다른 타입으로 데이터를 저장하는 예를 한국은행 데이터 파싱과정과 함께 정리한다.

### 작성이유

- 프로젝트 초반에는 REST API로부터 받아온 스트링으로 넘어온 숫자 데이터를 그대로 DB에 저장했다. 
- 그리고 숫자에 관련된 DB컬럼 또한 스트링타입이었다.

초반 빠른 개발로 결과물을 확인하기 위한 방식이었는데, 어느정도 정제가 필요한 지금은 DATA_VALUE 컬럼을 float 타입으로 지정하고자 한다.

- DATA_VALUE 컬럼을 float로 지정한 후에는 
- 크롤링코드에서 DataFrame으로 INSERT하는 로직을 변경해주어야 한다.

## 의존성 import

In [1]:
#-*- coding:utf-8 -*-

import psycopg2
import pandas.io.sql as pandas_sql
from pandas import DataFrame
from sqlalchemy import create_engine
import urllib3
import json

## database 접속정보 설정

In [2]:
dict_account = {
    'host': 'localhost',
    'dbname': 'stock_data',
    'user': 'postgres',
    'password': '1234'
}

conn_str = "host='{}' dbname='{}' user={} password={}"\
    .format(
        dict_account['host'],
        dict_account['dbname'],
        dict_account['user'],
        dict_account['password']
    )

alchemy_conn = create_engine(
    "postgresql://{}:{}@localhost:5432/{}"
        .format(
            dict_account['user'],
            dict_account['password'],
            dict_account['dbname']
        )
)

arr_columns = [
        'STAT_NAME',  'STAT_CODE',  'ITEM_CODE1', 'ITEM_CODE2', 'ITEM_CODE3',
        'ITEM_NAME1', 'ITEM_NAME2', 'ITEM_NAME3', 'DATA_VALUE', 'TIME'
    ]

dict_columns_type = {
    'STAT_NAME': 'str',
    'STAT_CODE': 'str',
    'ITEM_CODE1': 'str',
    'ITEM_CODE2': 'str',
    'ITEM_CODE3': 'str',
    'ITEM_NAME1': 'str',
    'ITEM_NAME2': 'str',
    'ITEM_NAME3': 'str',
    'DATA_VALUE': 'float',
    'TIME': 'str'
}

## 한국은행 api 접근
### !!!! 주의) API 키를 꼭!!! 직접 입력해야 정상동작한다.
### 여기서는 API 키 노출문제로 지워놓음

In [3]:
url = "http://ecos.bok.or.kr/api/StatisticSearch/[api 키 입력!!]/json/kr/1/5000/064Y001/DD/20140101/20191231/0001000"

http = urllib3.PoolManager()
ret = http.request("GET", url, headers={'Content-Type': 'application/json'})

str_response = ret.data.decode('utf-8')
dict_data = json.loads(str_response)
arr_data = dict_data['StatisticSearch']['row']

In [4]:
# arr_data

## pandas dataframe 내의 컬럼 데이터 타입 지정
  
- **키워드 인자 dtype 을 명시적으로 지정하는 방식**  
https://cmdlinetips.com/2018/09/how-to-change-data-type-for-one-or-more-columns-in-pandas-dataframe/  
python 3.x 또는 pandas 버전에서 deprecated 된것으로 보인다.
  
  
- **DataFrame이 생성된 후 column의 타입을 명시적으로 변경하는 방식**  
해외자료  
https://github.com/pandas-dev/pandas/issues/9287  
https://stackoverflow.com/questions/41517273/how-to-change-pandas-category-type-using-a-dtype-dict  
국내자료  
https://riptutorial.com/ko/pandas/example/10052/dtype-%EB%B3%80%EA%B2%BD%ED%95%98%EA%B8%B0

### 1) 2차원 배열로 변환
[dict1, dict2, dict3, .... dicti, .... dictn]  
의 방식은 통하지 않는다.  
  
따라서 2차원 배열의 형식으로 변환을 해주어야 한다.

In [5]:
# [[getattr(i,j) for j in arr_columns] for i in arr_data]
# 이상하게도 python 3.x 에서 getattr이 제대로 동작하지 않는다.

[[ dict_data[column_nm] for column_nm in arr_columns] for dict_data in arr_data]

# for i in arr_data:
#     for j in arr_columns:
#         print(i[j])

# for i in arr_data:
#     print(i['STAT_NAME'])

# arr_data[0]['STAT_NAME']

[['6.1.1 주식시장(일별)',
  '064Y001',
  '0001000',
  ' ',
  ' ',
  'KOSPI지수',
  ' ',
  ' ',
  '1967.19',
  '20140102'],
 ['6.1.1 주식시장(일별)',
  '064Y001',
  '0001000',
  ' ',
  ' ',
  'KOSPI지수',
  ' ',
  ' ',
  '1946.14',
  '20140103'],
 ['6.1.1 주식시장(일별)',
  '064Y001',
  '0001000',
  ' ',
  ' ',
  'KOSPI지수',
  ' ',
  ' ',
  '1953.28',
  '20140106'],
 ['6.1.1 주식시장(일별)',
  '064Y001',
  '0001000',
  ' ',
  ' ',
  'KOSPI지수',
  ' ',
  ' ',
  '1959.44',
  '20140107'],
 ['6.1.1 주식시장(일별)',
  '064Y001',
  '0001000',
  ' ',
  ' ',
  'KOSPI지수',
  ' ',
  ' ',
  '1958.96',
  '20140108'],
 ['6.1.1 주식시장(일별)',
  '064Y001',
  '0001000',
  ' ',
  ' ',
  'KOSPI지수',
  ' ',
  ' ',
  '1946.11',
  '20140109'],
 ['6.1.1 주식시장(일별)',
  '064Y001',
  '0001000',
  ' ',
  ' ',
  'KOSPI지수',
  ' ',
  ' ',
  '1938.54',
  '20140110'],
 ['6.1.1 주식시장(일별)',
  '064Y001',
  '0001000',
  ' ',
  ' ',
  'KOSPI지수',
  ' ',
  ' ',
  '1948.92',
  '20140113'],
 ['6.1.1 주식시장(일별)',
  '064Y001',
  '0001000',
  ' ',
  ' ',
  'KOSPI지수',
  ' ',


### 2) 데이터 타입 변환

In [6]:
data_for_insert = [[ dict_data[column_nm] for column_nm in arr_columns] for dict_data in arr_data]

df_test = DataFrame(data_for_insert, columns=arr_columns)
df_test = df_test.astype(dtype=dict_columns_type)

### 3) 변환결과 확인

In [7]:
df_test.dtypes

STAT_NAME      object
STAT_CODE      object
ITEM_CODE1     object
ITEM_CODE2     object
ITEM_CODE3     object
ITEM_NAME1     object
ITEM_NAME2     object
ITEM_NAME3     object
DATA_VALUE    float64
TIME           object
dtype: object

## Dataframe 내에 api 결과 데이터 저장

In [8]:
df_kospi_insert = DataFrame(data_for_insert, columns=arr_columns)
df_kospi_insert = df_kospi_insert.astype(dtype=dict_columns_type)

## Dataframe 내의 컬럼 data type 변경

In [9]:
df_kospi_insert.dtypes

STAT_NAME      object
STAT_CODE      object
ITEM_CODE1     object
ITEM_CODE2     object
ITEM_CODE3     object
ITEM_NAME1     object
ITEM_NAME2     object
ITEM_NAME3     object
DATA_VALUE    float64
TIME           object
dtype: object

In [10]:
df_kospi_insert

,STAT_NAME,STAT_CODE,ITEM_CODE1,ITEM_CODE2,ITEM_CODE3,ITEM_NAME1,ITEM_NAME2,ITEM_NAME3,DATA_VALUE,TIME
0,6.1.1 주식시장(일별),064Y001,0001000,,,KOSPI지수,,,1967.19,20140102
1,6.1.1 주식시장(일별),064Y001,0001000,,,KOSPI지수,,,1946.14,20140103
2,6.1.1 주식시장(일별),064Y001,0001000,,,KOSPI지수,,,1953.28,20140106
3,6.1.1 주식시장(일별),064Y001,0001000,,,KOSPI지수,,,1959.44,20140107
4,6.1.1 주식시장(일별),064Y001,0001000,,,KOSPI지수,,,1958.96,20140108
...,...,...,...,...,...,...,...,...,...,...
1410,6.1.1 주식시장(일별),064Y001,0001000,,,KOSPI지수,,,2072.42,20191001
1411,6.1.1 주식시장(일별),064Y001,0001000,,,KOSPI지수,,,2031.91,20191002
1412,6.1.1 주식시장(일별),064Y001,0001000,,,KOSPI지수,,,2020.69,20191004
1413,6.1.1 주식시장(일별),064Y001,0001000,,,KOSPI지수,,,2021.73,20191007
